# PEFT 库 QLoRA 实战

通常，模型被量化后不会进一步训练用于下游任务，因为由于权重和激活的较低精度，训练可能不稳定。

但是由于PEFT方法只添加额外的可训练参数，这使得我们可以使用PEFT适配器（Adapter）来训练一个量化模型！将量化与PEFT结合起来可以成为在单个GPU上训练大模型的微调策略。

例如，`QLoRA` 是一种将模型量化为4位然后使用LoRA进行训练的方法，使得在单个16GB GPU（本教程以 NVIDIA T4为例）上微调一个具有65B参数的大模型成为可能。

THUDM Hugging Face 主页：https://huggingface.co/THUDM

## 教程说明

本教程使用 QLoRA 论文中介绍的量化技术：`NF4 数据类型`、`双量化` 和 `混合精度计算`，在 `ChatGLM3-6b` 模型上实现了 QLoRA 微调。并展示了完整的 QLoRA 微调流程，具体如下：

- 数据准备
    - 下载数据集
    - 设计 Tokenizer 函数处理样本（map、shuffle、flatten）
    - 自定义批量数据处理类 DataCollatorForChatGLM
- 训练模型
    - 加载 ChatGLM3-6B 量化模型
    - PEFT 量化模型预处理（prepare_model_for_kbit_training）
    - QLoRA 适配器配置（TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING）
    - 微调训练超参数配置（TrainingArguments）
    - 开启训练（trainer.train)
    - 保存QLoRA模型（trainer.model.save_pretrained)
- [模型推理](peft_chatglm_inference.ipynb)
    - 加载 ChatGLM3-6B 基础模型
    - 加载 ChatGLM3-6B QLoRA 模型（PEFT Adapter）
    - 微调前后对比

In [ ]:
import os

import torch
import torch.nn as nn
from transformers import GPT2Tokenizer, AutoConfig, OPTForCausalLM

model_id = "E:\model\language\opt-125m"

model = OPTForCausalLM.from_pretrained(model_id,trust_remote_code=True).half().cuda()

tokenizer = GPT2Tokenizer.from_pretrained(model_id, trust_remote_code=True)

In [ ]:
from peft import prepare_model_for_int8_training

model = prepare_model_for_int8_training(model)

In [ ]:
# 获取当前模型占用的 GPU显存（差值为预留给 PyTorch 的显存）
memory_footprint_bytes = model.get_memory_footprint()
memory_footprint_mib = memory_footprint_bytes / (1024 ** 3)  # 转换为 GB

print(f"{memory_footprint_mib:.2f}GB")

In [ ]:
model

In [ ]:
from peft import LoraConfig,get_peft_model
config=LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj", "fc_in", "fc_out"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model=get_peft_model(model,config)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Abirate/english_quotes")

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

In [ ]:

tokenized_dataset = dataset.map(lambda samples: tokenizer(samples["quote"]), batched=True)


In [ ]:
from transformers import DataCollatorForLanguageModeling

# 数据收集器，用于处理语言模型的数据，这里设置为不使用掩码语言模型(MLM)
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
from transformers import TrainingArguments, Trainer

model_dir = "models"

training_args = TrainingArguments(
        output_dir=f"{model_dir}/{model_id}-lora",  # 指定模型输出和保存的目录
        per_device_train_batch_size=4,  # 每个设备上的训练批量大小
        learning_rate=2e-4,  # 学习率
        fp16=True,  # 启用混合精度训练，可以提高训练速度，同时减少内存使用
        logging_steps=20,  # 指定日志记录的步长，用于跟踪训练进度
        max_steps=100, # 最大训练步长
        # num_train_epochs=1  # 训练的总轮数
    )

In [ ]:
# 添加 LoRA 模块后的模型
model

In [ ]:
trainer = Trainer(
    model=model,  # 指定训练时使用的模型
    train_dataset=tokenized_dataset["train"],  # 指定训练数据集
    args=training_args,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
model_path = f"{model_dir}/{model_id}-lora-int8"

#trainer.save_model(model_path)
model.save_pretrained(model_path)

In [ ]:
lora_model = trainer.model

In [ ]:
text = "Two things are infinite: "
inputs = tokenizer(text, return_tensors="pt").to(0)

out = lora_model.generate(**inputs, max_new_tokens=48)
print(tokenizer.decode(out[0], skip_special_tokens=True))   